# Optimizando los Parámetros del Modelo

Ahora que tenemos un modelo y datos, es tiempo de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. Entrenar un modelo es un proceso iterativo; en cada iteración el modelo hace una suposición sobre la salida, calcula el error en su suposición (*pérdida*), recolecta las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior), y **optimiza** estos parámetros usando descenso de gradiente. Para un recorrido más detallado de este proceso, revisa este video en [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Código Prerrequisito
Cargamos el código de las secciones anteriores en `Datasets & DataLoaders` y `Build Model`.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Obtener datos de entrenamiento y prueba
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hiperparámetros

Los hiperparámetros son parámetros ajustables que te permiten controlar el proceso de optimización del modelo. Diferentes valores de hiperparámetros pueden impactar el entrenamiento del modelo y las tasas de convergencia ([leer más sobre ajuste de hiperparámetros](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html))

Definimos los siguientes hiperparámetros para el entrenamiento:
 - **Número de Épocas** - el número de veces para iterar sobre el conjunto de datos
 - **Tamaño de Lote** - el número de muestras de datos propagadas a través de la red antes de que los parámetros se actualicen
 - **Tasa de Aprendizaje** - cuánto actualizar los parámetros del modelo en cada lote/época. Valores más pequeños resultan en una velocidad de aprendizaje lenta, mientras que valores grandes pueden resultar en un comportamiento impredecible durante el entrenamiento.


In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Bucle de Optimización

Una vez que establecemos nuestros hiperparámetros, podemos entrenar y optimizar nuestro modelo con un bucle de optimización. Cada iteración del bucle de optimización se llama una **época**. 

Cada época consiste de dos partes principales:
 - **El Bucle de Entrenamiento** - itera sobre el conjunto de datos de entrenamiento e intenta converger a parámetros óptimos.
 - **El Bucle de Validación/Prueba** - itera sobre el conjunto de datos de prueba para verificar si el rendimiento del modelo está mejorando.

Vamos a familiarizarnos brevemente con algunos de los conceptos usados en el bucle de entrenamiento. Adelante, salta para ver la [Implementación Completa](#implementación-completa) del bucle de optimización.

### Función de Pérdida

Cuando se presenta con algunos datos de entrenamiento, nuestro modelo no entrenado probablemente no dará la respuesta correcta. **Función de pérdida** mide el grado de disimilitud del resultado obtenido al valor objetivo, y es la función de pérdida lo que queremos minimizar durante el entrenamiento. Para calcular la pérdida sumamos las predicciones de nuestro modelo en los puntos de datos dados y las comparamos contra los valores objetivo reales.

Las funciones de pérdida comunes incluyen [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Error Cuadrático Medio) para tareas de regresión, y [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Pérdida Logarítmica Negativa) para clasificación. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combina ``nn.LogSoftmax`` y ``nn.NLLLoss``.

Pasamos las predicciones de salida de nuestro modelo a ``nn.CrossEntropyLoss``, que normalizará las predicciones y calculará la pérdida.

In [ ]:
# Inicializar la función de pérdida
loss_fn = nn.CrossEntropyLoss()

### Optimizador

La optimización es el proceso de ajustar los parámetros del modelo para reducir el error del modelo en cada paso de entrenamiento. **Algoritmos de optimización** definen cómo se realiza este proceso (en este ejemplo usamos Descenso de Gradiente Estocástico). Toda la lógica de optimización está encapsulada en el objeto ``optimizer``. Aquí, usamos el optimizador SGD; además, hay muchos [diferentes optimizadores](https://pytorch.org/docs/stable/optim.html) disponibles en PyTorch tales como ADAM y RMSProp, que funcionan mejor para diferentes tipos de modelos y datos.

Inicializamos el optimizador registrando los parámetros del modelo que necesitan ser entrenados, y pasando el hiperparámetro de la tasa de aprendizaje.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Dentro del bucle de entrenamiento, la optimización sucede en tres pasos:
 * Llama a ``optimizer.zero_grad()`` para reiniciar los gradientes de los parámetros del modelo. Los gradientes por defecto se suman; para prevenir el doble conteo, los ponemos explícitamente en cero en cada iteración.
 * Retropropaga la pérdida de predicción con una llamada a ``loss.backward()``. PyTorch deposita los gradientes de la pérdida con respecto a cada parámetro.
 * Una vez que tenemos nuestros gradientes, llamamos a ``optimizer.step()`` para ajustar los parámetros por los gradientes recolectados en el paso hacia atrás.

## Implementación Completa
Definimos las funciones ``train_loop`` que recorre el bucle de optimización, y ``test_loop`` que evalúa el rendimiento del modelo contra nuestros datos de prueba.

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Calcular predicción y pérdida
        pred = model(X)
        loss = loss_fn(pred, y)

        # Retropropagación
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"pérdida: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Rendimiento de Prueba: \n Precisión: {(100*correct):>0.1f}%, Pérdida Promedio: {test_loss:>8f} \n")

Inicializamos la función de pérdida y optimizador, y los pasamos a ``train_loop`` y ``test_loop``.
Siéntete libre de incrementar el número de épocas para rastrear el rendimiento de mejora del modelo.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Época {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("¡Terminado!")

---

## Lectura Adicional
- [Función de Pérdida](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)
- [Tutorial de Warmstart Training](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)